# <font color='blue'>ALPAR - Governo Digital - Processo Inteligente</font>

# <font color='red'>recomendação de ações - gravando em MySQL</font>

In [ ]:
# !pip install mysql-connector-python

In [ ]:
import pandas as pd
import numpy as np
import copy
import csv
import codecs
import time
from os.path import expanduser
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode

print("pandas versão", pd.__version__)
print("numpy versão", np.__version__)
print("csv versão", csv.__version__)

pd.options.display.max_rows = 2000
pd.options.display.width = 120
pd.options.display.max_colwidth = 100

# <font color='black'>trata o dataset total</font>

In [ ]:
Simulacao = True                            # alterna True ou False para tratar de dados reias ou simulados

# caminho dos arquivos...
home = expanduser("~")
caminho_in = './dados_gd/'
arq_tasks = caminho_in + ('n_tasks.csv' if Simulacao else 'tasks.csv')

df_tasks = pd.read_csv(arq_tasks, sep = ',', dtype = str)
print("linhas = %s , colunas = %s " % df_tasks.shape)

# df_tasks

In [ ]:
# exclui colunas não utilizáveis
lst_del = ['Entidade', 'Usuário', 'Grupo', 
           'Motivo de cancelamento', 'Notificação',
           'Comentário', 'Apoio', 'Data e Hora de atendimento',
           'Status externo', 'Data de agendamento', 'Agendamento',
           'Categoria', 'Grupo responsável', 'Prazo (em segundos)']
df_tasks.drop(lst_del, axis = 1, inplace = True)
lst_nomes = ['entidade', 'servico', 'protocolo', 'dt_conclusao', 'dt_criacao',
             'acao', 'encaminhado_para', 'encerrado', 'cancelado']
df_tasks.columns = lst_nomes

In [ ]:
# transforma datas do tipo string para tipo data
df_tasks['dt_conclusao']= pd.to_datetime(df_tasks['dt_conclusao'])
df_tasks['dt_criacao']= pd.to_datetime(df_tasks['dt_criacao'])

In [ ]:
# verifica a situação de cada protocolo
df = df_tasks[['protocolo', 'encerrado', 'cancelado']].copy()

df.loc[df['encerrado'] == 'false', 'encerrado'] = 0
df.loc[df['encerrado'] == 'true', 'encerrado'] = 1
df.loc[df['cancelado'] == 'false', 'cancelado'] = 0
df.loc[df['cancelado'] == 'true', 'cancelado'] = 1

df['encerrado'] = pd.to_numeric(df['encerrado'])
df['cancelado'] = pd.to_numeric(df['cancelado'])

# df

In [ ]:
# agrupa por protocolo, obtem o máximo de encerrado e cancelado
df = df.groupby('protocolo').agg({'encerrado': 'max', 'cancelado': 'max'})

In [ ]:
# filtra os protocolos cancelados
df.drop(df[df['cancelado'] == 1].index, inplace=True)

# exclui a coluna de protocolos cancelados
df.drop('cancelado', axis=1, inplace=True)

# cria um dataframe de processos encerrados
df_ence = df[(df['encerrado'] == 1)].copy()

# cria um dataframe de processos em andamento
df_anda = df[(df['encerrado'] == 0)].copy()

# exclui a coluna encerrado dos 2 dataframes
df_ence.drop('encerrado', axis=1, inplace=True)
df_anda.drop('encerrado', axis=1, inplace=True)

In [ ]:
print(df_ence.shape)
print(df_anda.shape)

In [ ]:
# cria o dataframe de encerrados e dataframe de em andamento
lst_ence = df_ence.index.tolist()
lst_anda = df_anda.index.tolist()

# <font color='black'>cria e trata o dataset de protocolos encerrados</font>

In [ ]:
# cria dataset de tasks encerradas
df_ence = df_tasks[df_tasks['protocolo'].isin(lst_ence)].copy()

In [ ]:
# coloca a string <vazio> nos valores NULL
lst_mod = list(df_ence[df_ence['encaminhado_para'].isnull()].index)
df_ence.loc[lst_mod, 'encaminhado_para'] = '<Vazio>'

In [ ]:
# ordena por entidade, serviço, protocolo, dt_criação
df_ence.sort_values(by = ['entidade', 'servico', 'protocolo', 'dt_criacao'], inplace = True)

# refaz o índice para a nova ordenação
df_ence.reset_index(drop = True, inplace = True)

In [ ]:
# calcula a quantidade de dias da ação e exclui as datas de criação e de conclusão
df_ence['dias'] = (df_ence['dt_conclusao'] - df_ence['dt_criacao']) / np.timedelta64(1, 'D')
df_ence.drop(['dt_conclusao', 'dt_criacao'], axis=1, inplace=True)

In [ ]:
# exclui colunas não utilizáveis
df_ence.drop(['encerrado', 'cancelado'], axis=1, inplace=True)

# df_ence

In [ ]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['protocolo', 'entidade', 'servico'])
      .agg(
        lst_acao=('acao', lambda x: tuple(x)),
        lst_encaminhado=('encaminhado_para', lambda x: tuple(x)),
        sum_dias=('dias', sum)).reset_index())

In [ ]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['entidade', 'servico', 'lst_acao', 'lst_encaminhado'])
      .agg(
        media_dias=('sum_dias', 'mean')).reset_index())

# <font color='black'>cria o dataset de protocolos em andamento</font>

In [ ]:
# cria dataset de tasks em andamento
df_anda = df_tasks[df_tasks['protocolo'].isin(lst_anda)].copy()

In [ ]:
# exclui colunas não utilizáveis
df_anda.drop(['encerrado', 'cancelado', 'dt_conclusao'], axis=1, inplace=True)

# exclui as linhas com a coluna ação NULL
df_anda = df_anda[~df_anda['acao'].isnull()].copy()

# df_anda

In [ ]:
# ordena por entidade, serviço, protocolo, dt_criação
df_anda.sort_values(by = ['entidade', 'servico', 'protocolo', 'dt_criacao'], inplace = True)

# refaz o índice para a nova ordenação
df_anda.reset_index(drop = True, inplace = True)

In [ ]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_anda = (df_anda.groupby(['protocolo', 'entidade', 'servico'])
      .agg(lst_acao=('acao', lambda x: tuple(x)))
      .reset_index()).copy()
df_anda.sort_values(['entidade', 'servico'], inplace = True)
# df_anda

# <font color='black'>gera estrutura de recomendações</font>

In [ ]:
start = time.process_time()
tot = 0

# analisa cada protocolo em andamento
# procura a sequência do protocolo em andamento para sugerir o restante da sequência

# zera a lista de recomendações
lst_reco = []
const_max_tempo = 999999999.9
tot_anda = range(len(df_anda))
# tot_anda = range(1000)
print('total de protocolos a processar =', tot_anda)

# para cada protocolo em andamento procura uma coleção de sequências dos protocolos encerrados
anda_quebra = [None, None]
for i in tot_anda:
    if anda_quebra != [df_anda.loc[i, 'entidade'], df_anda.loc[i, 'servico']]:
        anda_quebra = [df_anda.loc[i, 'entidade'], df_anda.loc[i, 'servico']]
        df_aux1 = df_ence[(df_ence['entidade'] == df_anda.loc[i, 'entidade']) & 
                          (df_ence['servico'] == df_anda.loc[i, 'servico'])]
    
    n_acoes = len(df_anda.loc[i, 'lst_acao'])
    df_aux = df_aux1[(df_aux1['lst_acao'].apply(lambda x: x[:n_acoes]) == df_anda.loc[i, 'lst_acao'])]
    
    df_aux.reset_index(inplace = True)
    
    # zera a variável que vai achar o registro "entidade/serviço" com o menor tempo de conclusão
    # para quando for achado mais de 1 possibilidade
    min_tempo = const_max_tempo
    dt = {'sequencia': [], 'encaminhamento': []}

    qtd_casos = len(df_aux)
    tot += qtd_casos
    
    for k in range(qtd_casos):
        valor = round(np.float(df_aux.loc[k, 'media_dias']), 6)
        if (valor < min_tempo) or (min_tempo == None):
            min_tempo = valor
            dt = {'sequencia': list(df_aux.loc[k, 'lst_acao']),
                  'encaminhamento': list(df_aux.loc[k, 'lst_encaminhado'])}
            
    indice = len(list(df_anda.loc[i, 'lst_acao'])) if dt else -1
    if qtd_casos == 1:
        tipo_recomendacao = 2
    elif qtd_casos > 1:
        tipo_recomendacao = 3
    else:
        tipo_recomendacao = 1
        
    it = {'protocolo': df_anda.loc[i, 'protocolo'],
            'seq_atual': list(df_anda.loc[i, 'lst_acao']),
            'recomendacao': dt,
            'tempo_medio': None if min_tempo == const_max_tempo else min_tempo,
            'reco_acao_idx': indice,
            'qtd_casos' : qtd_casos,
            'tipo_recomendacao' : tipo_recomendacao}
    lst_reco.append(it)
    if i % 100 == 0:
        print(i, tot)

print(i, tot)
print('tempo =', time.process_time() - start)

# <font color='black'>cria um arquivo com recomendações</font>

## <font color='green'>define os SCRIPTS das tabelas de recomendações para o MySQL</font>

In [ ]:
script_tb_recomenda = (
"CREATE TABLE `tb_recomenda` ("
  "`sg_acao`               int NOT NULL AUTO_INCREMENT,"
  "`nu_seq`                smallint NOT NULL,"
  "`fk_protocolo`          varchar(21) NOT NULL,"
  "`nm_acao`               varchar(128) NOT NULL,"
  "`nm_encaminhado`        varchar(128) NOT NULL,"
  "`fg_proximo`            tinyint NOT NULL,"
  "PRIMARY KEY (`sg_acao`),"
  "KEY `fk_protocolo_idx` (`fk_protocolo`),"
  "CONSTRAINT `fk_protocolo` FOREIGN KEY (`fk_protocolo`) REFERENCES `tb_protocolo` (`pk_protocolo`)"
")"
)
script_tb_protocolo = (
"CREATE TABLE `tb_protocolo` ("
  "`pk_protocolo`          varchar(21) NOT NULL,"
  "`tp_medio`              decimal(18,6) DEFAULT NULL,"
  "`nm_prox_acao`          varchar(128) DEFAULT NULL,"
  "`nm_prox_enc`           varchar(128) DEFAULT NULL,"
  "`dt_timestamp`          timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,"
  "`nu_qtd_casos`          int NOT NULL,"
  "`cd_tipo_reco`          smallint NOT NULL,"
  "PRIMARY KEY (`pk_protocolo`)"
")"
)
tabelas = (('tb_protocolo', script_tb_protocolo), ('tb_recomenda', script_tb_recomenda))

## <font color='green'>CRIA as LISTAS de gravação de protocolos e recomendações no MySQL</font>

In [ ]:
# prepara uma lista de tuplas para gravação da tabela "tb_protocolo" no MySQL
lst_protocolo = []
lst_recomenda = []
for k in lst_reco:
    prx_acao = None
    prx_encaminhamento = None
    
    n_acoes_ence = len(k['recomendacao']['sequencia'])
    for j in range(n_acoes_ence):
        if j == k['reco_acao_idx']:
            proximo = 1
            prx_acao = k['recomendacao']['sequencia'][j]
            prx_encaminhamento = k['recomendacao']['encaminhamento'][j]
        else:
            proximo = 0
        
        lst_recomenda.append((j, 
                              k['protocolo'], 
                              k['recomendacao']['sequencia'][j], 
                              k['recomendacao']['encaminhamento'][j], 
                              proximo
                             )
        )

    lst_protocolo.append((k['protocolo'], 
                          k['tempo_medio'], 
                          prx_acao, 
                          prx_encaminhamento,
                          k['qtd_casos'],
                          k['tipo_recomendacao']
                         )
    )

## <font color='green'>LIMPA as tabelas de protocolos e recomendações no MySQL</font>

In [ ]:
My_host = 'localhost'
My_db = 'bd_testes'
My_user = 'gd'
My_pw = 'Alpar@123'

try:
    connection = mysql.connector.connect(host = My_host, database = My_db, user = My_user, password = My_pw)
    cursor = connection.cursor()
    
    # excluir tabelas
    for tb in reversed(tabelas):
        table_name = tb[0]
        table_script = tb[1]
        try:
            print("dropando tabela {}: \n".format(table_name), end='')
            apaga = xx = 'DROP TABLE IF EXISTS `' + My_db + '`.`' + table_name + '`;'
            cursor.execute(apaga)
        except mysql.connector.Error as err:
            msg = 'já existe' if err.errno == errorcode.ER_TABLE_EXISTS_ERROR else err.msg
        else:
            print("OK")

    # incluir tabelas
    for tb in tabelas:
        table_name = tb[0]
        table_script = tb[1]
        try:
            print("criando tabela {}: \n".format(table_name), end='')
            cursor.execute(table_script)
        except mysql.connector.Error as err:
            print(err.errno, err.msg)
        else:
            print("OK")

except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

## <font color='green'>GRAVA as tabelas de protocolos e recomendações no MySQL</font>

In [ ]:
try:
    connection = mysql.connector.connect(host = My_host, database = My_db, user = My_user, password = My_pw)

    insert_protocolo = (
        "INSERT INTO tb_protocolo "
        "(pk_protocolo, tp_medio, nm_prox_acao, nm_prox_enc, nu_qtd_casos, cd_tipo_reco)"
        "VALUES (%s, %s, %s, %s, %s, %s)"
    )
    insert_recomenda = (
        "INSERT INTO tb_recomenda "
        "(nu_seq, fk_protocolo, nm_acao, nm_encaminhado, fg_proximo)"
        "VALUES (%s, %s, %s, %s, %s)"
    )
    
    cursor_p = connection.cursor()
    cursor_r = connection.cursor()


    cursor_p.executemany (insert_protocolo, lst_protocolo)
    cursor_r.executemany (insert_recomenda, lst_recomenda)

    connection.commit()

    print(cursor_p.rowcount, "Record inserted successfully into tb_protocolo table")
    print(cursor_r.rowcount, "Record inserted successfully into tb_recomenda table")


except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        cursor_p.close()
        cursor_r.close()
        connection.close()
        print("MySQL connection is closed")

# <font color='black'>TESTES</font>